In [46]:
import numpy as np
import pandas as pd

In [167]:
def Hawkes_in(mu,alpha,beta,T,ite):
    # mu: the predicted zonal trip arrivals' rate at zone at previous time slot
    n=1
    num=[]
    while n<ite: # since hawkes is a stochastic process, the result is random with ranges. so we simulate n times then get the average value
        T=60 #set time interval as 60 min, so the mu should be no.s/min
        #time period set for simulation: Time interval
        #initiaization:
        s = np.array([0]) #set s initial value as zero, checkin time (t or tp). last event time.
        r = np.array([mu])
        t = np.random.exponential(1/r[0])
        U = np.random.random_sample()
        rnew = mu + np.sum(alpha*np.exp(-beta*(t-s[0])))
        #print (t,T,rnew/r[-1],U)
        while t<T and rnew/r[-1]>=U:
            #print (s)
            rnew = mu + sum(alpha*np.exp(-beta*(t-s)))#hawk process equation
            s=np.append(s,t)
            r=np.append(r,rnew)
    #generate a new event
            delta_t = np.random.exponential(1/rnew)
            t = t+delta_t #this is the new t: event time boundary
            U = np.random.random_sample()
        num.append(len(s)-1)
        n=n+1
    num=round(np.mean(num))
    return num

In [170]:
df=pd.read_pickle('firehoseTAZ')

In [171]:
df.head()

,time,cate,TAZ
0,2017-01-01 00:02:56,Food,95
1,2017-01-01 00:07:38,Professional & Other Places,95
2,2017-01-01 00:19:20,Food,95
3,2017-01-01 00:19:25,Food,95
4,2017-01-01 00:21:18,Nightlife Spot,95


In [172]:
df['hour_of_day'] = df['time'].dt.hour

In [192]:
n=318 # n is the taz number
t=24 # t is the hour of day
checkin=np.zeros([n,t])

In [193]:
for i in range(len(df)):
    taz=df.iloc[i][2]-1
    h=df.iloc[i][3]-1
    checkin[taz][h]+=1

In [198]:
alpha=numpy.load('alpha.npy') #calibrated alpha
beta=numpy.load('beta.npy') #calibrated beta
delta=numpy.load('delta.npy') #calibrated delta
gamma=numpy.load('gamma.npy') #calibrated delta
T=60
ite=1000

In [199]:
trips=numpy.load('wp.npy') # for warm up period
for i in range(n):
    for t in range(1,t):
        trips[i][t]=delta[i][t]*Hawkes_in(trips[i][t-1]/T,alpha[i][t],beta[i][t],T,ite)+delta[i][t]*gamma[i][t]*checkin[i][t]

### trips is the predicted trip arrival